In [2]:
texts_raw = '''
this is good
real bad
I like what happned
i hated that person
cats have 4 legs
I have shorted the stock, cuz i see a slowdown in business
'''

texts = texts_raw.split('\n')
texts = [i for i in texts if i != '']
print(texts)

['this is good', 'real bad', 'I like what happned', 'i hated that person', 'cats have 4 legs', 'I have shorted the stock, cuz i see a slowdown in business']


# Loading 300k tweets and getting sentiment on each one then saving that to CSV.


In [1]:
import pandas as pd

# Load 300K Tweets zm-6-2019-04-01-2022-12-11-all-zm-tweets.jsonl
file_path = 'zm-6-2019-04-01-2022-12-11-all-zm-tweets.jsonl' #full file. 
# file_path = 'zm-tweets-jan to april 2020.csv' # testing smaller file
# file_path = 'zm_mismatch_df.csv'  # testing on mismatch only 5k
# df = pd.read_csv(file_path)
df = pd.read_json(file_path, lines=True)
# tweet_zm_df = pd.read_json(tweet_json, lines=True)
# tweet_zm_df.head(10)
# df = pd.read_json(file_path)




In [10]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301098 entries, 0 to 301097
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   _type             301098 non-null  object             
 1   url               301098 non-null  object             
 2   date              301098 non-null  datetime64[ns, UTC]
 3   content           301098 non-null  object             
 4   renderedContent   301098 non-null  object             
 5   id                301098 non-null  int64              
 6   user              301098 non-null  object             
 7   replyCount        301098 non-null  int64              
 8   retweetCount      301098 non-null  int64              
 9   likeCount         301098 non-null  int64              
 10  quoteCount        301098 non-null  int64              
 11  conversationId    301098 non-null  int64              
 12  lang              301098 non-null  object   

In [12]:
# temp --- getting 5k tweets 

# # these 2 lines works well (need to convert to list)
# texts = df['clean_tweet']
# texts = list(texts)


texts = df['content']
texts = list(texts)


In [13]:

len(texts)

301098

In [ ]:

# clean tweets and create a new col clean_tweets

# filter tweets that have cashtags that are 3 or lower (will remove alot of the spam and news report tweets)

# count remaining tweets.

In [14]:
# WORKING WITH CUDA and adding 300k tweets to test

# code wtih cuda reporting.

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:" if cuda_available else "CUDA not available")

# Load the RoBERTa tokenizer and model
model_name = f'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cuda" if cuda_available else "cpu")

# Create a list of 17,000 sample texts (replace this with your actual data)
# texts = ["Sample text {}".format(i) for i in range(17000)] #commented out to use my own text

# Set the batch size (you may need to adjust this based on your GPU memory)
batch_size = 32

# Function to classify sentiment in a batch of texts
def classify_sentiment(texts_batch):
    print(texts_batch)
    encoded_input = tokenizer(texts_batch, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {key: value.to("cuda" if cuda_available else "cpu") for key, value in encoded_input.items()}
    with torch.no_grad():
        logits = model(**encoded_input).logits
    probabilities = torch.softmax(logits, dim=1)
    print(probabilities)
    # labels = ["NEGATIVE" if p < 0.5 else "POSITIVE" for p in probabilities[:, 1]]
    max_indices = torch.argmax(probabilities, dim=1)
    # Map indices to labels
    labels_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    labels = [labels_map[index.item()] for index in max_indices]
    return labels
    # return probabilities

# Process the texts in batches
sentiments = []
num_batches = len(texts) // batch_size + int(len(texts) % batch_size > 0)

for i in range(0, len(texts), batch_size):
    texts_batch = texts[i:i + batch_size]
    sentiments_batch = classify_sentiment(texts_batch)
    sentiments.extend(sentiments_batch)
    
    # Print progress
    batches_completed = i // batch_size + 1
    batches_left = num_batches - batches_completed
    print(f"Batch {batches_completed}/{num_batches} completed. Batches left: {batches_left}")

# Print the first 10 sentiment labels
print(sentiments[:50])



c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available:


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['$ROKU $ARKK $TWLO $AMZN $DIS $NVAX $NVDA $AMD $MDB $DDOG $IWM $PYPL $SQ $SHOP $PTON $NOW $SNAP $ZM $NIO $CHWY $SPOT $NKE $TTD $UBER $ABNB $GME $TUP $BLNK $RIOT chopped around for months while going sideways before destruction\n\n$TSLA $AAPL making a super choppy bases now as well', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wood https://t.co/9q4dcY4yLt', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wood https://t.co/nG1UdnVT4N', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wood https://t.co/ucHouBneQP', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wood https://t.co/YG17kYRoWD', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wood https://t.co/2OVmAcM6Pb', '$ROKU $SQ $TSLA $ZM - 5 Investment Opportunities With Over 700% Upside, According to Cathie Wo

In [1]:
df['sentiment_r_latest'] = sentiments

NameError: name 'sentiments' is not defined

In [26]:
# df.to_csv('mismatch_w_r_latest_sentiment.csv')
df.to_csv('full300k_w_r_latest_sentiment_no_clean_no_finetuning.csv')

In [ ]:
# Add sentiment back into main df


# save main 300ktweets_with_sentiment to csv.

In [15]:
import torch
print(torch.version.cuda)

11.3
